# Project Summary

In [1]:
from game import Game
from investigate_game import InvestigateGame
from random_player import RandomPlayer
from min_max import MinMaxPlayer, AlphaBetaMinMaxPlayer

In [2]:
player1 = MinMaxPlayer(player_id=0, depth=2, symmetries=False)
player2 = AlphaBetaMinMaxPlayer(player_id=1, depth=3, symmetries=False)
game = InvestigateGame(Game())
winner = game.play(player1, player2, max_steps_draw=10)

-- BEGINNING OF THE GAME --
⬜️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️⬜️
Player ❌ chose to move (0, 0) to the bottom
⬜️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️⬜️
❌⬜️⬜️⬜️⬜️
Player ⭕️ chose to move (0, 0) to the bottom
⬜️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️⬜️
❌⬜️⬜️⬜️⬜️
⭕️⬜️⬜️⬜️⬜️
Player ❌ chose to move (0, 0) to the bottom
⬜️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️⬜️
❌⬜️⬜️⬜️⬜️
⭕️⬜️⬜️⬜️⬜️
❌⬜️⬜️⬜️⬜️
Player ⭕️ chose to move (1, 0) to the bottom
⬜️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️⬜️
❌⬜️⬜️⬜️⬜️
⭕️⬜️⬜️⬜️⬜️
❌⭕️⬜️⬜️⬜️
Player ❌ chose to move (0, 0) to the right
⬜️⬜️⬜️⬜️❌
⬜️⬜️⬜️⬜️⬜️
❌⬜️⬜️⬜️⬜️
⭕️⬜️⬜️⬜️⬜️
❌⭕️⬜️⬜️⬜️
Player ⭕️ chose to move (0, 0) to the bottom
⬜️⬜️⬜️⬜️❌
❌⬜️⬜️⬜️⬜️
⭕️⬜️⬜️⬜️⬜️
❌⬜️⬜️⬜️⬜️
⭕️⭕️⬜️⬜️⬜️
Player ❌ chose to move (0, 0) to the bottom
❌⬜️⬜️⬜️❌
⭕️⬜️⬜️⬜️⬜️
❌⬜️⬜️⬜️⬜️
⭕️⬜️⬜️⬜️⬜️
❌⭕️⬜️⬜️⬜️
Player ⭕️ chose to move (2, 0) to the bottom
❌⬜️⬜️⬜️❌
⭕️⬜️⬜️⬜️⬜️
❌⬜️⬜️⬜️⬜️
⭕️⬜️⬜️⬜️⬜️
❌⭕️⭕️⬜️⬜️
Player ❌ chose to move (0, 2) to the right
❌⬜️⬜️⬜️❌
⭕️⬜️⬜️⬜️⬜️
⬜️⬜️⬜️⬜️❌
⭕️⬜️⬜️⬜️⬜️
❌⭕️⭕️⬜️⬜️
Player ⭕️ chose to move (0, 2) to the top
⭕️⬜️⬜️